<a href="https://colab.research.google.com/github/havihaviplants/Machine-learning/blob/main/CIFAR_100%20%EC%99%84%EC%84%B1%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
import torchvision.models as models

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def load_data(train_path, test_path):

    train_data = np.load(train_path)
    x_train, y_train = train_data['data'], train_data['target']

    test_data = np.load(test_path)
    x_test = test_data['data']

    # 데이터 형태를 [배치 크기, 높이, 너비, 채널]로 변경
    x_train = x_train.reshape(-1, 48, 48, 3).astype('float32') / 255.0
    x_test = x_test.reshape(-1, 48, 48, 3).astype('float32') / 255.0

    # PyTorch 텐서로 변환 (차원 순서 조정)
    x_train = torch.tensor(x_train).permute(0, 3, 1, 2)
    x_test = torch.tensor(x_test).permute(0, 3, 1, 2)

    return x_train, torch.tensor(y_train, dtype=torch.long), x_test


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def train_model(model, train_loader, criterion, optimizer, epochs=20):
    for epoch in range(epochs):
        model.train()  # 훈련 모드로 설정
        running_loss = 0.0
        correct = 0
        total = 0

        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

    return model

def predict_model(model, test_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
    return predictions

   #  x_train, y_train, x_test = load_data('/content/gdrive/My Drive/cifar100_noisy_train.npz', '/content/gdrive/My Drive/cifar100_noisy_test_public.npz')

In [5]:
def main():
    x_train, y_train, x_test = load_data('/content/drive/MyDrive/MY Drive/cifar100_noisy_train.npz', '/content/drive/MyDrive/MY Drive/cifar100_noisy_test_public.npz')

    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)

    test_dataset = TensorDataset(x_test, torch.zeros(len(x_test)))
    test_loader = DataLoader(test_dataset, batch_size=24, shuffle=False)

    # 사전 훈련된 ResNet 모델 로드
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

    # CIFAR-100에 맞게 마지막 FC 레이어 수정
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 100)  # 100개 클래스에 맞게 조정

    model = model.to(device)

    # 학습률
    optimizer = optim.Adam(model.parameters(), lr=0.0002)
    criterion = nn.CrossEntropyLoss()

    # 모델 훈련
    train_model(model, train_loader, criterion, optimizer)

    # 모델 평가
    predictions = predict_model(model, test_loader)

    # 제출 파일 생성
    submission = pd.DataFrame({
        'Id': np.arange(len(x_test)),
        'Category': predictions
    })
    submission.to_csv('my_resnet_submission.csv', index=False)

if __name__ == "__main__":
    main()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 88.7MB/s]


Epoch 1/20, Loss: 3.2981, Accuracy: 21.75%
Epoch 2/20, Loss: 2.6688, Accuracy: 32.66%
Epoch 3/20, Loss: 2.3709, Accuracy: 38.54%
Epoch 4/20, Loss: 2.1174, Accuracy: 43.89%
Epoch 5/20, Loss: 1.8789, Accuracy: 49.03%
Epoch 6/20, Loss: 1.6312, Accuracy: 54.87%
Epoch 7/20, Loss: 1.3955, Accuracy: 60.57%
Epoch 8/20, Loss: 1.1598, Accuracy: 66.51%
Epoch 9/20, Loss: 0.9494, Accuracy: 71.94%
Epoch 10/20, Loss: 0.7743, Accuracy: 76.71%
Epoch 11/20, Loss: 0.6362, Accuracy: 80.59%
Epoch 12/20, Loss: 0.5257, Accuracy: 83.67%
Epoch 13/20, Loss: 0.4620, Accuracy: 85.53%
Epoch 14/20, Loss: 0.3983, Accuracy: 87.50%
Epoch 15/20, Loss: 0.3673, Accuracy: 88.45%
Epoch 16/20, Loss: 0.3303, Accuracy: 89.64%
Epoch 17/20, Loss: 0.3118, Accuracy: 90.13%
Epoch 18/20, Loss: 0.2912, Accuracy: 90.77%
Epoch 19/20, Loss: 0.2829, Accuracy: 91.02%
Epoch 20/20, Loss: 0.2521, Accuracy: 92.01%
